In [1]:
%%writefile mapper.py

import sys
import re
from imp import reload
if sys.version[0] == '2':
    reload(sys)
    sys.setdefaultencoding("utf-8")
    
for line in sys.stdin:
    try:
        #print ("line", line)
        article_id, text = unicode(line.strip()).split('\t', 1)
    except ValueError as e:
        print("error", e)
        continue
    #text = re.sub("^\W+|\W+$", "", text, flags=re.UNICODE)
    words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
    #print (words)
    for word in words:
        #print (word)
        print >> sys.stderr, "reporter:counter:Wiki stats,Total words,%d" % 1
        #print "%s\t%s" % (word.lower(), 1)
        print "%s\t%d" % (word.lower(), 1)

Overwriting mapper.py


In [2]:
#! python -m pycompile mapper.py

In [3]:
%%writefile reducer.py

import sys

current_key = None
word_sum = 0

Overwriting reducer.py


In [4]:
%%writefile -a reducer.py

for line in sys.stdin:
    try:
        key, count = line.strip().split('\t', 1)
        count = int(count)
    except ValueError as e:
        continue
    if current_key != key:
        if current_key:
            print "%s\t%d" % (current_key, word_sum)
        word_sum = 0
        current_key = key
    word_sum += count

if current_key:
    print "%s\t%d" % (current_key, word_sum)

Appending to reducer.py


In [5]:
! hdfs dfs -ls /data/wiki

Found 1 items
drwxrwxrwx   - jovyan supergroup          0 2018-08-01 17:15 /data/wiki/en_articles_part


In [6]:
%%bash

OUT_DIR="wordcount_result_"$(date +"%s%6N")
NUM_REDUCERS=8

hdfs dfs -rm -r -skipTrash ${OUT_DIR} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.job.name="Streaming wordCount" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files mapper.py,reducer.py \
    -mapper "python mapper.py" \
    -combiner "python reducer.py" \
    -reducer "python reducer.py" \
    -input /data/wiki/en_articles_part \
    -output ${OUT_DIR} > /dev/null

hdfs dfs -cat ${OUT_DIR}/part-00000 | head

0%however	1
0&\mathrm{if	1
0(8)320-1234	1
0)).(1	2
0,03	1
0,1,...,n	1
0,1,0	1
0,1,\dots,n	1
0,5	1
0,50	1


rm: `wordcount_result_1545182911798629': No such file or directory
18/12/19 01:28:34 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/12/19 01:28:34 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/12/19 01:28:34 INFO mapred.FileInputFormat: Total input files to process : 1
18/12/19 01:28:35 INFO mapreduce.JobSubmitter: number of splits:2
18/12/19 01:28:35 INFO Configuration.deprecation: mapred.job.name is deprecated. Instead, use mapreduce.job.name
18/12/19 01:28:35 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1545181678310_0005
18/12/19 01:28:35 INFO impl.YarnClientImpl: Submitted application application_1545181678310_0005
18/12/19 01:28:35 INFO mapreduce.Job: The url to track the job: http://3bfb327c519e:8088/proxy/application_1545181678310_0005/
18/12/19 01:28:35 INFO mapreduce.Job: Running job: job_1545181678310_0005
18/12/19 01:28:40 INFO mapreduce.Job: Job job_1545181678310_0005 running in uber mode : false
18/12/19 01:2

### Grade system output: 
===== Testing (num. 700): STARTING =====
Executing notebook with kernel: python2
Testing (num. 700): test CRS700_4 failed on line "0-00-655138-6	1"!
CRS700_4 description: Check the result word

===== Testing (num. 700): SUMMARY =====
Tests passed: crs700_3 crs700_5 mrb17 mrb19_0 mrb19_1 sp7_10 res1 res2_1 res3 res6_1 res7_1
Tests failed: crs700_4
==================================================
Duration: 57.0 sec
VCoreSeconds: 225 sec 

0